使用nvidia-smi命令来查看显卡信息

In [1]:
!nvidia-smi

Mon Sep 18 22:56:55 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 527.99       Driver Version: 527.99       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   34C    P0    23W /  95W |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

计算设备

在PyTorch中，CPU和GPU可以用torch.device('cpu') 和torch.device('cuda')表示。 应该注意的是，cpu设备意味着所有物理CPU和内存， 这意味着PyTorch的计算将尝试使用所有CPU核心。</BR> 
然而，GPU设备只代表一个卡和相应的显存。 如果有多个GPU，我们使用torch.device(f'cuda:{i}') 来表示第i块GPU（i从0开始）。 另外，cuda:0和cuda是等价的。

In [4]:
import torch
from torch import nn

torch.device('cpu'), torch.device('cuda'), torch.device('cuda:0')

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=0))

可以查询可用gpu的数量

In [3]:
torch.cuda.device_count()

1

定义了两个方便的函数， 这两个函数允许我们在请求的GPU不存在的情况下运行代码

In [5]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')


def try_all_gpus():  
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]"""
    devices = [torch.device(f'cuda:{i}')
               for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device('cpu')]


try_gpu(), try_gpu(10), try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

## 张量与GPU
可以查询张量所在的设备。 默认情况下，张量是在CPU上创建的。

In [6]:
x = torch.tensor([0, 1, 2, 3])
x.device

device(type='cpu')

存储在GPU上

In [7]:
X = torch.ones(2, 3, device=try_gpu())
X

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

假设我们至少有两个GPU，下面的代码将在第二个GPU上创建一个随机张量

In [8]:
Y = torch.rand(2, 3, device=try_gpu(1))
Y

tensor([[0.6365, 0.3148, 0.3580],
        [0.8440, 0.8762, 0.1143]])

In [ ]:
!nvidia-smi

如果我们要计算X + Y，我们需要决定在哪里执行这个操作

In [10]:
Z = Y.cuda(0)
print(Y)
print(Z)

tensor([[0.6365, 0.3148, 0.3580],
        [0.8440, 0.8762, 0.1143]])
tensor([[0.6365, 0.3148, 0.3580],
        [0.8440, 0.8762, 0.1143]], device='cuda:0')


现在数据在同一个GPU上（Z和X都在），我们可以将它们相加。

In [11]:
X+Z

tensor([[1.6365, 1.3148, 1.3580],
        [1.8440, 1.8762, 1.1143]], device='cuda:0')

假设变量Z已经存在于第1个GPU上。 如果我们还是调用Z.cuda(0)会发生什么？ 它将返回Z，而不会复制并分配新内存

In [13]:
Z.cuda(0) is Z

True

## 神经网络与GPU

In [14]:
net = nn.Sequential(nn.Linear(3, 1))
net = net.to(device=try_gpu())

net(X)

tensor([[-0.5918],
        [-0.5918]], device='cuda:0', grad_fn=<AddmmBackward0>)

确认模型参数存储在同一个GPU上

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)

只要所有的数据和参数都在同一个设备上， 我们就可以有效地学习模型